In [2]:
from qiskit import QuantumCircuit, execute
from qiskit import IBMQ, Aer
from qiskit.visualization import plot_histogram
from qiskit.providers.aer.noise import NoiseModel

# Build noise model from backend properties
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-melbourne', group='uom-researchers', project='default')
backend = provider.get_backend('ibmq_toronto')
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model
basis_gates = noise_model.basis_gates

/home/jvillanueva/.local/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [3]:
import numpy as np
from qiskit import execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Kraus, SuperOp
#from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_histogram

# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import thermal_relaxation_error
import qiskit.providers.aer.noise as noise

In [4]:
#Get backend properties
backend_properties = backend.properties()

In [5]:
#Thermal relaxation errors from backend properties
t1_avg = np.mean([backend_properties.t1(i) for i in range(26)])
t2_avg = np.mean([backend_properties.t2(i) for i in range(26)])
print("Average T1: ", t1_avg)
print("Average T2: ", t2_avg)

Average T1:  0.00010225869013834383
Average T2:  0.00013013891728065382


In [6]:
#Get backend configuration to determine CX connectivity
backend_configuration = backend.configuration()
connected_qubits = backend_configuration.coupling_map

#Get average CX error rate
cx_error_avg = 0
for qubit in connected_qubits:
    cx_error_avg += backend_properties.gate_error(gate = 'cx', qubits = qubit)
    print("Connected qubits:", qubit, "CX Error:", backend_properties.gate_error(gate = 'cx', qubits = qubit))
cx_error_avg /= len(connected_qubits)
print("Avg CX_error:", cx_error_avg)

Connected qubits: [0, 1] CX Error: 0.005442546740439119
Connected qubits: [1, 0] CX Error: 0.005442546740439119
Connected qubits: [1, 2] CX Error: 0.007616444248237542
Connected qubits: [1, 4] CX Error: 0.007370258687757647
Connected qubits: [2, 1] CX Error: 0.007616444248237542
Connected qubits: [2, 3] CX Error: 0.00871294762265129
Connected qubits: [3, 2] CX Error: 0.00871294762265129
Connected qubits: [3, 5] CX Error: 0.012736538609730563
Connected qubits: [4, 1] CX Error: 0.007370258687757647
Connected qubits: [4, 7] CX Error: 0.008425895682895812
Connected qubits: [5, 3] CX Error: 0.012736538609730563
Connected qubits: [5, 8] CX Error: 0.011911347309269543
Connected qubits: [6, 7] CX Error: 0.005715248207175588
Connected qubits: [7, 4] CX Error: 0.008425895682895812
Connected qubits: [7, 6] CX Error: 0.005715248207175588
Connected qubits: [7, 10] CX Error: 0.00728370160847891
Connected qubits: [8, 5] CX Error: 0.011911347309269543
Connected qubits: [8, 9] CX Error: 0.0135031489873

In [7]:
average_gate_errors = {}
#Get average gate error for each single qubit gate
for gate in basis_gates:
    if gate == 'cx' or gate == 'reset':
        pass
    else:
        gate_error = 0
        for qubit in range(27):
            gate_error += backend_properties.gate_error(gate = gate, qubits = qubit)
            print("Gate:", gate, "Qubit:", qubit, "Gate Error:", backend_properties.gate_error(gate = gate, qubits = qubit))
        gate_error /= 27
        average_gate_errors[gate] = gate_error
        print("Average Gate error:", gate_error, '\n')

print(average_gate_errors)

Gate: id Qubit: 0 Gate Error: 0.00026547794460338495
Gate: id Qubit: 1 Gate Error: 0.00016801223654374275
Gate: id Qubit: 2 Gate Error: 0.0004468978480111801
Gate: id Qubit: 3 Gate Error: 0.0005927857657485302
Gate: id Qubit: 4 Gate Error: 0.0014221356749439325
Gate: id Qubit: 5 Gate Error: 0.0005142426952746345
Gate: id Qubit: 6 Gate Error: 0.00020996476399993283
Gate: id Qubit: 7 Gate Error: 0.00023659746036134015
Gate: id Qubit: 8 Gate Error: 0.00026806759028301624
Gate: id Qubit: 9 Gate Error: 0.0003545146160285052
Gate: id Qubit: 10 Gate Error: 0.0002414717977513344
Gate: id Qubit: 11 Gate Error: 0.00023838392590461489
Gate: id Qubit: 12 Gate Error: 0.00030072416429430525
Gate: id Qubit: 13 Gate Error: 0.00016655596973940434
Gate: id Qubit: 14 Gate Error: 0.00019518866210253973
Gate: id Qubit: 15 Gate Error: 0.0011206840664690302
Gate: id Qubit: 16 Gate Error: 0.0003737267608056933
Gate: id Qubit: 17 Gate Error: 0.0003534621577780213
Gate: id Qubit: 18 Gate Error: 0.00203264225212

In [8]:
readout_errors = noise.device.basic_device_readout_errors(backend_properties)
avg_readout_errors = np.array([])
for i in range(26):
    avg_readout_errors = np.append(avg_readout_errors, (readout_errors[i][1].probabilities[0][1]+readout_errors[i][1].probabilities[1][0])/2)
avg_read_err = np.mean(avg_readout_errors)
print("Average readout Err:", avg_read_err)

Average readout Err: 0.04875384615384614


In [10]:
noise_model = NoiseModel(basis_gates = basis_gates)

#Depolarizing error on single qubit gates
for gate in average_gate_errors:
    depol_err = depolarizing_error(average_gate_errors[gate] * 0.1, 1, standard_gates = False) 
    noise_model.add_all_qubit_quantum_error(depol_err, [gate])

#Depolarizing error on two-qubit gates
depol_err_2 = depolarizing_error(cx_error_avg * 0.1, 2)
noise_model.add_all_qubit_quantum_error(depol_err_2, ['cx'])

#Add measurement error
meas_error_prob = avg_read_err * 0.1
meas_error = pauli_error([('X',meas_error_prob), ('I', 1 - meas_error_prob)])
noise_model.add_all_qubit_quantum_error(meas_error, ['measure'])

In [16]:
import json
import pickle

In [14]:
average_gate_errors_file = open("average_gate_errors.json", "w+")
json.dump(average_gate_errors, average_gate_errors_file)
average_gate_errors_file.close()

In [21]:
vals = [cx_error_avg, avg_read_err]
with open('error_vals.pkl', 'wb') as f:
    pickle.dump(vals, f)

In [22]:
with open('basis_gates.pkl', 'wb') as f:
    pickle.dump(basis_gates, f)

In [24]:
avg_read_err

0.05083846153846155